In [85]:
import pandas as pd
import numpy as np

from nltk import word_tokenize

In [86]:
data = pd.read_json("data.json")
data

,id,vac_name,desc,values
0,10,Электромонтажник,Обязанности: - Обеспечивать бесперебойную и на...,"[{'skill': 'металлообработка', 'start': 121, '..."
1,11,Менеджер по продаже по недвижимости,Обязанности: Работа по заявкам для своих сотру...,"[{'skill': 'Уверенный пользователь ПК', 'start..."
2,12,Супервайзер мерчендайзеров,«OPEN Group» одна из крупнейших российских ком...,"[{'skill': 'Управление командой', 'start': 289..."
3,13,Продавец-консультант,"Ты наш человек, если понятия «честность», «отк...","[{'skill': 'честность', 'start': 30, 'end': 39..."
4,14,Мастер маникюра,"В связи с расширением штата, салон премиум кла...","[{'skill': 'доброжелательность', 'start': 135,..."
...,...,...,...,...
789,4191,JavaScript программист (игры),Ищем в команду программиста на проект гемблинг...,[{'skill': 'совместно с командой разрабатывать...
790,4192,Специалист по работе с клиентами,"Ты наш человек, если понятия «честность», «отк...","[{'skill': 'честность', 'start': 30, 'end': 39..."
791,4193,Руководитель департамента складской логистики,Обязанности: -Осуществляет руководство произво...,[{'skill': 'Осуществляет руководство производс...
792,4194,Охранник,"Работа в крупной, яркой компании BKC-Internati...",[{'skill': 'Охрана помещения и материальных це...


In [87]:
def create_bio_tagged_data(id, vac_name, desc, values):

    tokenized_desc = word_tokenize(desc.lower(), language="russian")
    tokenized_vals = list(map(lambda x: word_tokenize(x["skill"].lower(), language="russian"), values))

    new_data = []
    i = 0
    while i < len(tokenized_desc):
        token = tokenized_desc[i]
        tag = 'O' 
        
        for skill_tokens in tokenized_vals:
            if tokenized_desc[i:i+len(skill_tokens)] == skill_tokens:
                tag = 'B-SKILL'
                new_data.append((id, vac_name, token, tag))
                for j in range(1, len(skill_tokens)):
                    i += 1
                    token = tokenized_desc[i]
                    tag = 'I-SKILL'
                    new_data.append((id, vac_name, token, tag))
                break 
                
        if tag == 'O': 
            new_data.append((id, vac_name, token, tag))
        i += 1

    return pd.DataFrame(new_data, columns=['id', 'position', 'token', 'tag'])

def apply_to_row(row):
    return create_bio_tagged_data(row['id'], row['vac_name'], row['desc'], row['values'])

In [88]:
tagged_dataframes = [apply_to_row(row) for index, row in data.iterrows()]
result_df = pd.concat(tagged_dataframes).reset_index(drop=True)
result_df.tail()

,id,position,token,tag
132119,4195,Продавец-кассир в кафе,зарплата,O
132120,4195,Продавец-кассир в кафе,от,O
132121,4195,Продавец-кассир в кафе,27,O
132122,4195,Продавец-кассир в кафе,тысяч,O
132123,4195,Продавец-кассир в кафе,рублей,O


In [89]:
result_df

,id,position,token,tag
0,10,Электромонтажник,обязанности,O
1,10,Электромонтажник,:,O
2,10,Электромонтажник,-,O
3,10,Электромонтажник,обеспечивать,O
4,10,Электромонтажник,бесперебойную,O
...,...,...,...,...
132119,4195,Продавец-кассир в кафе,зарплата,O
132120,4195,Продавец-кассир в кафе,от,O
132121,4195,Продавец-кассир в кафе,27,O
132122,4195,Продавец-кассир в кафе,тысяч,O


In [91]:
result_df.to_csv("test_data.csv", encoding="utf-16", index=False)

In [68]:
for_peft = data.copy()

for_peft["desc"] = "###User:\n" + for_peft["desc"].str.lower() + "\n\n"
for_peft["values"] = "###Bot:\n" + for_peft["values"].apply(lambda x: ";".join(list(map(lambda y: y["skill"].lower(), x))))
for_peft["text"] = for_peft["desc"] + for_peft["values"]
for_peft

,id,vac_name,desc,values,text
0,10,Электромонтажник,###User:\nобязанности: - обеспечивать беспереб...,###Bot:\nметаллообработка;сборке и монтаже эле...,###User:\nобязанности: - обеспечивать беспереб...
1,11,Менеджер по продаже по недвижимости,###User:\nобязанности: работа по заявкам для с...,###Bot:\nуверенный пользователь пк;умение обуч...,###User:\nобязанности: работа по заявкам для с...
2,12,Супервайзер мерчендайзеров,###User:\n«open group» одна из крупнейших росс...,###Bot:\nуправление командой;контроль мерченда...,###User:\n«open group» одна из крупнейших росс...
3,13,Продавец-консультант,"###User:\nты наш человек, если понятия «честно...",###Bot:\nчестность;открытость;добросовестность...,"###User:\nты наш человек, если понятия «честно..."
4,14,Мастер маникюра,"###User:\nв связи с расширением штата, салон п...",###Bot:\nдоброжелательность;коммуникабельность...,"###User:\nв связи с расширением штата, салон п..."
...,...,...,...,...,...
789,4191,JavaScript программист (игры),###User:\nищем в команду программиста на проек...,###Bot:\nсовместно с командой разрабатывать кл...,###User:\nищем в команду программиста на проек...
790,4192,Специалист по работе с клиентами,"###User:\nты наш человек, если понятия «честно...",###Bot:\nчестность;открытость;добросовестность...,"###User:\nты наш человек, если понятия «честно..."
791,4193,Руководитель департамента складской логистики,###User:\nобязанности: -осуществляет руководст...,###Bot:\nосуществляет руководство производстве...,###User:\nобязанности: -осуществляет руководст...
792,4194,Охранник,"###User:\nработа в крупной, яркой компании bkc...",###Bot:\nохрана помещения и материальных ценно...,"###User:\nработа в крупной, яркой компании bkc..."


In [70]:
print(for_peft["text"][10])

###User:
о нас: рпк \ником реклама\"- лидер на российском рынке производства рекламы. за 24 года успешной работы \"ником реклама\" удостоена звания генеральных партнёров у крупнейших компаний страны, а так же ежегодно занимает призовые места на международных профессиональных конкурсах. в настоящий момент, компания осуществляет свою деятельность на территории всей российской федерации, имеет собственное, мощное и оснащённое современной техникой производство, а так же собственный обширный транспортный цех. в связи с расширением штата сотрудников рассмотрим на конкурсной основе кандидатов на позицию «мастер цеха». чем предстоит заниматься:  организация и контроль эффективной работы участков производства: распределение и выдача заданий по участкам, при необходимости ссылка на рабочую документацию и файлы фрезеровки, печати; обеспечение бесперебойной работы участков: проверка наличия необходимых материалов на складе по проектам; отслеживание технологии изготовления объектов наружной рекламы

In [80]:
for_peft.to_csv("for_peft.csv", encoding="utf-16", index=False)

In [81]:
pd.read_csv("for_peft.csv", encoding="utf-16")

,id,vac_name,desc,values,text
0,10,Электромонтажник,###User:\nобязанности: - обеспечивать беспереб...,###Bot:\nметаллообработка;сборке и монтаже эле...,###User:\nобязанности: - обеспечивать беспереб...
1,11,Менеджер по продаже по недвижимости,###User:\nобязанности: работа по заявкам для с...,###Bot:\nуверенный пользователь пк;умение обуч...,###User:\nобязанности: работа по заявкам для с...
2,12,Супервайзер мерчендайзеров,###User:\n«open group» одна из крупнейших росс...,###Bot:\nуправление командой;контроль мерченда...,###User:\n«open group» одна из крупнейших росс...
3,13,Продавец-консультант,"###User:\nты наш человек, если понятия «честно...",###Bot:\nчестность;открытость;добросовестность...,"###User:\nты наш человек, если понятия «честно..."
4,14,Мастер маникюра,"###User:\nв связи с расширением штата, салон п...",###Bot:\nдоброжелательность;коммуникабельность...,"###User:\nв связи с расширением штата, салон п..."
...,...,...,...,...,...
789,4191,JavaScript программист (игры),###User:\nищем в команду программиста на проек...,###Bot:\nсовместно с командой разрабатывать кл...,###User:\nищем в команду программиста на проек...
790,4192,Специалист по работе с клиентами,"###User:\nты наш человек, если понятия «честно...",###Bot:\nчестность;открытость;добросовестность...,"###User:\nты наш человек, если понятия «честно..."
791,4193,Руководитель департамента складской логистики,###User:\nобязанности: -осуществляет руководст...,###Bot:\nосуществляет руководство производстве...,###User:\nобязанности: -осуществляет руководст...
792,4194,Охранник,"###User:\nработа в крупной, яркой компании bkc...",###Bot:\nохрана помещения и материальных ценно...,"###User:\nработа в крупной, яркой компании bkc..."


In [82]:
import os

def convert_to_utf8(input_file, output_file):
    with open(input_file, 'r', encoding='utf-16') as f_in:
        content = f_in.read()
    
    with open(output_file, 'w', encoding='utf-8') as f_out:
        f_out.write(content)

In [83]:
# Пример использования
input_file = 'train.csv'  
output_file = 'train_utf8.csv' 

convert_to_utf8(input_file, output_file)

In [84]:
pd.read_csv("train_utf8.csv")

,id,vac_name,desc,values,text
0,10,Электромонтажник,###User:\nобязанности: - обеспечивать беспереб...,###Bot:\nметаллообработка;сборке и монтаже эле...,###User:\nобязанности: - обеспечивать беспереб...
1,11,Менеджер по продаже по недвижимости,###User:\nобязанности: работа по заявкам для с...,###Bot:\nуверенный пользователь пк;умение обуч...,###User:\nобязанности: работа по заявкам для с...
2,12,Супервайзер мерчендайзеров,###User:\n«open group» одна из крупнейших росс...,###Bot:\nуправление командой;контроль мерченда...,###User:\n«open group» одна из крупнейших росс...
3,13,Продавец-консультант,"###User:\nты наш человек, если понятия «честно...",###Bot:\nчестность;открытость;добросовестность...,"###User:\nты наш человек, если понятия «честно..."
4,14,Мастер маникюра,"###User:\nв связи с расширением штата, салон п...",###Bot:\nдоброжелательность;коммуникабельность...,"###User:\nв связи с расширением штата, салон п..."
...,...,...,...,...,...
789,4191,JavaScript программист (игры),###User:\nищем в команду программиста на проек...,###Bot:\nсовместно с командой разрабатывать кл...,###User:\nищем в команду программиста на проек...
790,4192,Специалист по работе с клиентами,"###User:\nты наш человек, если понятия «честно...",###Bot:\nчестность;открытость;добросовестность...,"###User:\nты наш человек, если понятия «честно..."
791,4193,Руководитель департамента складской логистики,###User:\nобязанности: -осуществляет руководст...,###Bot:\nосуществляет руководство производстве...,###User:\nобязанности: -осуществляет руководст...
792,4194,Охранник,"###User:\nработа в крупной, яркой компании bkc...",###Bot:\nохрана помещения и материальных ценно...,"###User:\nработа в крупной, яркой компании bkc..."
